In [19]:
#导入相关包
import numpy as np
import pandas as pd
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import EarlyStopping
#创建模型 -待完成
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adam
import time
import os

In [20]:
#函数预处理模块

#读取文件路径为入参的csv文件。返回DataFram类型的 文件内容
def mLoad_Data_Fun(mFileNamePara_str:str)->pd.DataFrame:
    mOrgData_pd = pd.read_csv(mFileNamePara_str)#'D:\\mMLP\\StockData\\mdata0.csv')
    return mOrgData_pd


#数据预处理   高、开、低、收,成交量 的数据有意义的是其相对于昨日的涨跌幅
def mPreDeal_OneData_Fun(mParaPrice_array:np.array)->np.array:
    mDataTemp_array = []
    i = 1
    while(i<len(mParaPrice_array)):
        #因为最大涨跌幅是10%，所以*10 将数据限制在-1-1间
        mData_D = (mParaPrice_array[i] / mParaPrice_array[i-1] - 1) * 10
        mDataTemp_array.append(mData_D)
        i += 1
    mDataTemp_array = np.array(mDataTemp_array)
    return mDataTemp_array


#数据预处理  高-收、收-开、开-低和昨日收盘价格 的比率=
def mUpDownShiti_Percent_Fun(mHighPara_array:np.array,mOpenPara_array:np.array,mLowPara_array:np.array,\
                             mClosePara_array:np.array,):
    mUpPercentTemp_np = []
    mDownPercentTemp_np = []
    mShitiPercentTemp_np = []
    
    i = 1
    while(i < len(mClosePara_array)):
        
        #因为 这3个值的最大值为 20%(涨跌幅10%，天地板时)，所以* 5 将其限制在 0-1之间
        if mOpenPara_array[i] > mClosePara_array[i]:
            #上影线
            mUpPercentTemp_d = (mHighPara_array[i] - mOpenPara_array[i]) / mClosePara_array[i-1] * 5
            #下影线
            mDownPercentTemp_d = (mClosePara_array[i] - mLowPara_array[i]) / mClosePara_array[i-1] * 5
        else:
            #上影线
            mUpPercentTemp_d = (mHighPara_array[i] - mClosePara_array[i]) / mClosePara_array[i-1] * 5
            #下影线
            mDownPercentTemp_d = (mOpenPara_array[i] - mLowPara_array[i]) / mClosePara_array[i-1] * 5
            
        #实体 因为 开、收的大小关系 本身也含有信息，所以不取其绝对值
        mShitiPercentTemp_d = (mOpenPara_array[i] - mClosePara_array[i]) / mClosePara_array[i-1] * 5
            
        mUpPercentTemp_np.append(mUpPercentTemp_d)
        mDownPercentTemp_np.append(mDownPercentTemp_d)
        mShitiPercentTemp_np.append(mShitiPercentTemp_d)
        
        i += 1
        
    mUpPercentTemp_np = np.array(mUpPercentTemp_np)
    mDownPercentTemp_np = np.array(mDownPercentTemp_np)
    mShitiPercentTemp_np = np.array(mShitiPercentTemp_np)
    return mUpPercentTemp_np,mDownPercentTemp_np,mShitiPercentTemp_np



#判断类别y的预处理 
def mPreDeal_Category_Y_Fun(mClosePara_array:np.array)->np.array:
    """
    按 往后5天内，close的涨跌幅划分区间
    0：跌幅大于3个点的 
    1：跌幅在 1.5 - 0 之间的
    2：涨幅在 0 - 1.5之间的
    3：涨幅在 1.5 -3 之间的
    4： 涨幅大于 3个点的
    
    """
    mReturn_array = []
    i = 1
    while(i<len(mClosePara_array)):
        #如果后续天数不足5天，则取后续剩余天数为判断天数
        if(i+4 < len(mClosePara_array)):
            mCloseTemp_np = mClosePara_array[i:i+5]
        else:
            mCloseTemp_np = mClosePara_array[i:]
        
        mMaxClose_f = np.max(mCloseTemp_np)
        #将张地方转为百分点新式，即3 表示张 3个百分点
        mRate = ( mMaxClose_f / mClosePara_array[i-1]  -1 ) * 100 
        
        if mRate > 3:
            mReturn_array.append(4)
        elif mRate > 1.5:
            mReturn_array.append(3)
        elif mRate > 0:
            mReturn_array.append(2)
        elif mRate > -1.5:
            mReturn_array.append(1)
        else:
            mReturn_array.append(0)
        
        i += 1
    
    mReturn_array = np.array(mReturn_array)
    
    return mReturn_array




In [21]:
#数据总预处理-将数据处理成
def mPreDeal_Total_Fun(mData_pd:pd.DataFrame):
    mCloseTemp_array = np.array(mData_pd.loc[:,'close'])
    mOpenTemp_array = np.array(mData_pd.loc[:,'open'])
    mHighTemp_array = np.array(mData_pd.loc[:,'high'])
    mLowTemp_array = np.array(mData_pd.loc[:,'low'])
    mVolumeTemp_array = np.array(mData_pd.loc[:,'volume'])
    
    #忘了RSI指标的意义，暂时先搁置
    #mRSI_array = np.array(mData_pd.loc[1:,'RSI'])
    
    #对高、开、低、收、成交量 进行预处理，获取其相对于其昨日数据的比率
    mPreCloseSimple_array = mPreDeal_OneData_Fun(mCloseTemp_array)
    mPreOpenSimple_array = mPreDeal_OneData_Fun(mOpenTemp_array)
    mPreHighSimple_array = mPreDeal_OneData_Fun(mHighTemp_array)
    mPreLowSimple_array = mPreDeal_OneData_Fun(mLowTemp_array)
    mPreVolume_array = mPreDeal_OneData_Fun(mVolumeTemp_array)
    
    #获取上下影线、实体相对于昨日收盘价的比率
    mUpPercent_np,mDownPercent_np,mShitiPercent_np = mUpDownShiti_Percent_Fun(mHighTemp_array,mOpenTemp_array,mLowTemp_array,\
                                                                              mCloseTemp_array)
  
    
    mPreFinalY_list = mPreDeal_Category_Y_Fun(mCloseTemp_array)
    
    #7:mRSI_array,
    mPreFinalData_pd = {0:mPreCloseSimple_array, 1:mPreOpenSimple_array, 2:mPreHighSimple_array , 3:mPreLowSimple_array \
                    , 4:mPreVolume_array ,5:mUpPercent_np,6:mDownPercent_np,7:mShitiPercent_np,'y':mPreFinalY_list}
    mPreFinalData_pd = pd.DataFrame(mPreFinalData_pd)
    
    return mPreFinalData_pd


In [22]:
"""
mOrgData_df = mLoad_Data_Fun('D:\\mMLP\\StockData\\mTrainStock\\600056.csv')
mPreData_df = mPreDeal_Total_Fun(mOrgData_df)
mPreData_df.head()
"""

"\nmOrgData_df = mLoad_Data_Fun('D:\\mMLP\\StockData\\mTrainStock\\600056.csv')\nmPreData_df = mPreDeal_Total_Fun(mOrgData_df)\nmPreData_df.head()\n"

In [23]:
#数据预处理 -将mPreDeal_Total_Fun预处理后的数据整理成RNN模型的输入类型
def mPre_Change_RNNPara_Fun(mPreFinalPara_pd:pd.DataFrame, mStepPara_I=10):
    #
    x = []
    y = []
    mRangeLen_I = len(mPreFinalPara_pd)-mStepPara_I 
    for i in range(mRangeLen_I):
        mDataTemp = np.array(mPreFinalPara_pd.loc[i:i+mStepPara_I-1,:].drop('y',axis=1)) 
        x.append(mDataTemp) 
        y.append(mPreFinalPara_pd.loc[i+mStepPara_I-1,'y']) 
        

    x = np.array(x)
    y = pd.Series(y)
    
    return x,y

In [24]:
"""
mPreDataRnn_x,mPreDataRnn_y = mPre_Change_RNNPara_Fun(mPreData_df,20)
print(mPreDataRnn_x.shape,mPreDataRnn_y.shape)
print(mPreDataRnn_x[1])
"""

'\nmPreDataRnn_x,mPreDataRnn_y = mPre_Change_RNNPara_Fun(mPreData_df,20)\nprint(mPreDataRnn_x.shape,mPreDataRnn_y.shape)\nprint(mPreDataRnn_x[1])\n'

In [25]:

#转化为one-hot类型
def mYChange_One_hot_Fun(mYTrainPara_list:pd.Series):
    y_train = np_utils.to_categorical(mYTrainPara_list, 5)
    return y_train

In [26]:
"""
#将Y转为one-hot向量
mPreDataRnnOne_y = mYChange_One_hot_Fun(mPreDataRnn_y)

#训练、测试集分离
mXTrain_np = mPreDataRnn_x[:-800]
mYTrain = mPreDataRnnOne_y[:-800]

mXTest_np = mPreDataRnn_x[-800:-500]
mYTest_np = mPreDataRnnOne_y[-800:-500]

mXShape = mXTrain_np[0].shape
mYshape = mYTrain[0].shape
"""

'\n#将Y转为one-hot向量\nmPreDataRnnOne_y = mYChange_One_hot_Fun(mPreDataRnn_y)\n\n#训练、测试集分离\nmXTrain_np = mPreDataRnn_x[:-800]\nmYTrain = mPreDataRnnOne_y[:-800]\n\nmXTest_np = mPreDataRnn_x[-800:-500]\nmYTest_np = mPreDataRnnOne_y[-800:-500]\n\nmXShape = mXTrain_np[0].shape\nmYshape = mYTrain[0].shape\n'

In [27]:
#搭建RNN模型
#--------------------2Rnn2MLP_tanh_softmax_10_5_5_4_SGD_0.001_
def mCreate_Rnn_Model_Fun(mRnnXDataShape,mRnnYshape):
    #优化器选择与设置
    #mOptimizer_obj = RMSprop(learning_rate=0.001)
    mOptimizer_obj =Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    mRnnModel = Sequential()
    mRnnModel.add(SimpleRNN(units = 20,input_shape=(mRnnXDataShape[0],mRnnXDataShape[1]),activation='tanh'\
                            ,name='mFirst_Rnn_layer'))#,return_sequences='True'
    
    #mRnnModel.add(SimpleRNN(units = 10,activation='sigmoid',name='mSecond_Rnn_layer'))#sigmoid
    
    mRnnModel.add(Dense(units=10,activation='tanh',name='mFourth_MLP_layers'))
    
    mRnnModel.add(Dense(units=mRnnYshape[0],activation='softmax',name='mSecond_MLP_layers'))
    
    mRnnModel.summary()
    
    
    mRnnModel.compile(loss='categorical_crossentropy',optimizer=mOptimizer_obj,metrics=['accuracy'])
    
    return mRnnModel


In [28]:
"""
mRnnModel_model = mCreate_Rnn_Model_Fun(mXShape,mYshape) 
    
#对分类为3 重点关注
mClaseWeight_dic = {0:2.0, 1:1.0 , 2: 1.0 , 3:2.0 , 4:2.0}
    
early_stopping = EarlyStopping(monitor='accuracy', patience=20)
#,callbacks=[early_stopping]
mTrainResult_obj = mRnnModel_model.fit(mXTrain_np,mYTrain,class_weight=mClaseWeight_dic,epochs=1000,batch_size=150,\
                                       validation_data =(mXTest_np,mYTest_np),callbacks=[early_stopping]);
"""

"\nmRnnModel_model = mCreate_Rnn_Model_Fun(mXShape,mYshape) \n    \n#对分类为3 重点关注\nmClaseWeight_dic = {0:2.0, 1:1.0 , 2: 1.0 , 3:2.0 , 4:2.0}\n    \nearly_stopping = EarlyStopping(monitor='accuracy', patience=20)\n#,callbacks=[early_stopping]\nmTrainResult_obj = mRnnModel_model.fit(mXTrain_np,mYTrain,class_weight=mClaseWeight_dic,epochs=1000,batch_size=150,                                       validation_data =(mXTest_np,mYTest_np),callbacks=[early_stopping]);\n"

In [29]:
def mGet_RNN_FinalPreDate_Fun(mStockPath_str):
    #数据读取
    mOrgData_df = mLoad_Data_Fun(mStockPath_str)
    #数据预处理，主要是数据的归一化
    mPreData_df = mPreDeal_Total_Fun(mOrgData_df)
    #数据预处理，将数据转为RNN神经网络的输入
    mPreDataRnn_x,mPreDataRnn_y = mPre_Change_RNNPara_Fun(mPreData_df,20)

    #将Y转为one-hot向量
    mPreDataRnnOne_y = mYChange_One_hot_Fun(mPreDataRnn_y)

    
    #训练、测试集分离
    #测试集所占比率
    mTestRate_D = 0.2
    mTestLocation = int(len(mPreDataRnn_x) * mTestRate_D)
    
    print(mTestLocation)
    mXTrain_np = mPreDataRnn_x[:-mTestLocation]
    mYTrain_np = mPreDataRnnOne_y[:-mTestLocation]

    mXTest_np = mPreDataRnn_x[-mTestLocation:]
    mYTest_np = mPreDataRnnOne_y[-mTestLocation:]
    
    return mXTrain_np,mYTrain_np,mXTest_np,mYTest_np


In [30]:
#一支股票的训练'D:\\mMLP\\StockData\\mTrainStock\\600056.csv'
def mRNN_Train_OneStock_Fun(mStockDataPath_str):
    #数据读取
    mXTrain_np,mYTrain_np,mXTest_np,mYTest_np = mGet_RNN_FinalPreDate_Fun(mStockDataPath_str)

    mXShape = mXTrain_np[0].shape
    mYshape = mYTrain_np[0].shape


    mRnnModel_model = mCreate_Rnn_Model_Fun(mXShape,mYshape) 

    #对分类为3 重点关注
    mClaseWeight_dic = {0:2.0, 1:1.0 , 2: 1.0 , 3:2.0 , 4:2.0}

    early_stopping = EarlyStopping(monitor='accuracy', patience=10)
    #,callbacks=[early_stopping],class_weight=mClaseWeight_dic,
    mTrainResult_obj = mRnnModel_model.fit(mXTrain_np,mYTrain_np,epochs=1000,batch_size=100,\
                                       validation_data =(mXTest_np,mYTest_np),callbacks=[early_stopping]);
    mAccuray = mTrainResult_obj.history['val_accuracy']
    if mAccuray[-1] > 0.5:
        mSymbol_str = mStockDataPath_str[-10:-4]
        print(mSymbol_str,' accuracy ',mAccuray[-1])
        mNeedWriteContent_str = mSymbol_str + ' accuracy ' + str(mAccuray[-1]) + '\n'
        mRecordFile_str = 'D:\\mMLP\\StockData\\mResult\\mSatisfy.txt'
        
        with open(mRecordFile_str,'a+') as mSave_obj:
            mSave_obj.write(mNeedWriteContent_str)

In [33]:
def mRnn_Train_AllStock_Fun():
    mFilePath_str = 'E:\\mMachineLerning\\Own\\RNN\\mTrain_StockData\\'
    mSymbolList_np = os.listdir(mFilePath_str)
    
    i = 0
    while(i <len(mSymbolList_np)):
        mDataPath_str = 'E:\\mMachineLerning\\Own\\RNN\\mTrain_StockData\\' + mSymbolList_np[i][:6] + '.csv'
        print(i,'   ',mDataPath_str)
        mRNN_Train_OneStock_Fun(mDataPath_str)
        if i > 300:
            return
        i += 1
    #mRNN_Train_OneStock_Fun('D:\\mMLP\\StockData\\mTrainStock\\600820.csv')

In [34]:
mRnn_Train_AllStock_Fun()

0     E:\mMachineLerning\Own\RNN\mTrain_StockData\000012.csv
295
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mFirst_Rnn_layer (SimpleRNN) (None, 20)                580       
_________________________________________________________________
mFourth_MLP_layers (Dense)   (None, 10)                210       
_________________________________________________________________
mSecond_MLP_layers (Dense)   (None, 5)                 55        
Total params: 845
Trainable params: 845
Non-trainable params: 0
_________________________________________________________________


D:\Anaconda\envs\csdn_ml_env\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/1000
12/12 [==============================] - 14s 50ms/step - loss: 1.7474 - accuracy: 0.2448 - val_loss: 1.7036 - val_accuracy: 0.2712
Epoch 2/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.7066 - accuracy: 0.2139 - val_loss: 1.6359 - val_accuracy: 0.2847
Epoch 3/1000
12/12 [==============================] - 0s 4ms/step - loss: 1.6414 - accuracy: 0.2165 - val_loss: 1.5887 - val_accuracy: 0.2475
Epoch 4/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.5810 - accuracy: 0.2636 - val_loss: 1.5519 - val_accuracy: 0.2441
Epoch 5/1000
12/12 [==============================] - 0s 2ms/step - loss: 1.5604 - accuracy: 0.2710 - val_loss: 1.5207 - val_accuracy: 0.2915
Epoch 6/1000
12/12 [==============================] - 0s 2ms/step - loss: 1.5253 - accuracy: 0.3166 - val_loss: 1.4941 - val_accuracy: 0.3356
Epoch 7/1000
12/12 [==============================] - 0s 2ms/step - loss: 1.5131 - accuracy: 0.3152 - val_loss: 1.4674 - val_accuracy: 0.3729
Epoc

12/12 [==============================] - 0s 3ms/step - loss: 1.1903 - accuracy: 0.4814 - val_loss: 1.2176 - val_accuracy: 0.5085
Epoch 59/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.1490 - accuracy: 0.4994 - val_loss: 1.2247 - val_accuracy: 0.5085
Epoch 60/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.1435 - accuracy: 0.4997 - val_loss: 1.2195 - val_accuracy: 0.5051
Epoch 61/1000
12/12 [==============================] - 0s 3ms/step - loss: 1.1434 - accuracy: 0.4988 - val_loss: 1.2202 - val_accuracy: 0.4949
Epoch 62/1000
12/12 [==============================] - 0s 2ms/step - loss: 1.1209 - accuracy: 0.5129 - val_loss: 1.2238 - val_accuracy: 0.4949
Epoch 63/1000
12/12 [==============================] - 0s 2ms/step - loss: 1.1506 - accuracy: 0.4974 - val_loss: 1.2164 - val_accuracy: 0.5051
Epoch 64/1000
12/12 [==============================] - 0s 2ms/step - loss: 1.1564 - accuracy: 0.4935 - val_loss: 1.2222 - val_accuracy: 0.4915
Epoch 65/1000

12/12 [==============================] - 0s 2ms/step - loss: 1.0137 - accuracy: 0.5879 - val_loss: 1.2648 - val_accuracy: 0.4881
Epoch 116/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.9871 - accuracy: 0.5962 - val_loss: 1.2573 - val_accuracy: 0.4949
Epoch 117/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.9729 - accuracy: 0.5976 - val_loss: 1.2692 - val_accuracy: 0.4949
Epoch 118/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.9798 - accuracy: 0.6024 - val_loss: 1.2707 - val_accuracy: 0.4949
Epoch 119/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.9669 - accuracy: 0.6052 - val_loss: 1.2601 - val_accuracy: 0.5085
Epoch 120/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.9774 - accuracy: 0.5901 - val_loss: 1.3032 - val_accuracy: 0.4746
Epoch 121/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.9512 - accuracy: 0.5975 - val_loss: 1.3012 - val_accuracy: 0.4712
Epoch 1

Epoch 172/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.8076 - accuracy: 0.6796 - val_loss: 1.4612 - val_accuracy: 0.4712
Epoch 173/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8273 - accuracy: 0.6729 - val_loss: 1.4674 - val_accuracy: 0.4780
Epoch 174/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8428 - accuracy: 0.6745 - val_loss: 1.4563 - val_accuracy: 0.4780
Epoch 175/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.8232 - accuracy: 0.6698 - val_loss: 1.4400 - val_accuracy: 0.5051
Epoch 176/1000
12/12 [==============================] - 0s 4ms/step - loss: 0.8010 - accuracy: 0.6817 - val_loss: 1.4355 - val_accuracy: 0.5051
Epoch 177/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.7919 - accuracy: 0.6748 - val_loss: 1.4501 - val_accuracy: 0.4949
Epoch 178/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.7853 - accuracy: 0.6798 - val_loss: 1.4522 - val_accuracy:

Epoch 229/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.6741 - accuracy: 0.7366 - val_loss: 1.6412 - val_accuracy: 0.5017
Epoch 230/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.7045 - accuracy: 0.7078 - val_loss: 1.6426 - val_accuracy: 0.5051
Epoch 231/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.7048 - accuracy: 0.7160 - val_loss: 1.6454 - val_accuracy: 0.5017
Epoch 232/1000
12/12 [==============================] - 0s 2ms/step - loss: 0.6992 - accuracy: 0.7284 - val_loss: 1.6572 - val_accuracy: 0.5017
Epoch 233/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.7198 - accuracy: 0.7244 - val_loss: 1.6639 - val_accuracy: 0.4983
Epoch 234/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.6959 - accuracy: 0.7209 - val_loss: 1.6788 - val_accuracy: 0.4983
Epoch 235/1000
12/12 [==============================] - 0s 3ms/step - loss: 0.6843 - accuracy: 0.7302 - val_loss: 1.6561 - val_accuracy:

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\mMLP\\StockData\\mResult\\mSatisfy.txt'